### **Import packages, download datasets and weights**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
! git clone https://github.com/smitasasindran/MidasYoloPlanercnn.git

Cloning into 'MidasYoloPlanercnn'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 214 (delta 73), reused 186 (delta 47), pack-reused 0
Receiving objects: 100% (214/214), 14.30 MiB | 39.16 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [3]:
%cd MidasYoloPlanercnn
# ! ls

/content/MidasYoloPlanercnn


In [4]:
!ls

cfg					 models		     train.py	 wiki
data					 README.md	     transforms
dataloader				 testing_modules.py  utils
EVA5_Assignment15_MidasYolo_Train.ipynb  test.py	     weights


In [5]:
# Copy weights
!mkdir weights
!cp /content/gdrive/MyDrive/eva5/ass15/model-f6b98070.pt weights/
!cp /content/gdrive/'My Drive'/eva5/ass13/yolov3-spp-ultralytics.pt weights/
!cp /content/gdrive/'My Drive'/eva5/ass13/custom/weights/best15_a.pt weights/
!ls weights

mkdir: cannot create directory ‘weights’: File exists
best15_a.pt  model-f6b98070.pt	tempfile  yolov3-spp-ultralytics.pt


In [6]:
!mkdir /content/rawdata
%cd /content/rawdata
!ls

/content/rawdata


In [7]:
# # !cp /content/gdrive/'My Drive'/eva5/ass15a/planes/planes.zip rawdata/
!cp /content/gdrive/'My Drive'/eva5/ass15a/depth/depth.zip .
!cp /content/gdrive/'My Drive'/eva5/ass15a/images/images.zip . 
!cp /content/gdrive/'My Drive'/eva5/ass15a/labels/labels.zip .

! ls

depth.zip  images.zip  labels.zip


In [8]:
!unzip images.zip -d images
!unzip labels.zip #-d labels
!unzip depth.zip -d depth

# !unzip planes.zip -d planes
!ls


Streaming output truncated to the last 5000 lines.
  inflating: labels/a94.txt          
  inflating: labels/a9.txt           
  inflating: labels/aaf2bb4fc2.txt   
  inflating: labels/african-american-man-in-hardhat-and-safety-vest-picture-id690555096.txt  
  inflating: labels/african-american-man-with-cherry-picker-truck-picture-id637288204.txt  
  inflating: labels/africanamerican-woman-working-at-shipping-port-picture-id1036354066.txt  
  inflating: labels/african-male-demonstrating-against-air-pollution-with-mask-picture-id1190069899.txt  
  inflating: labels/Aimg_001.txt     
  inflating: labels/Aimg_006.txt     
  inflating: labels/Aimg_009.txt     
  inflating: labels/Aimg_015.txt     
  inflating: labels/Aimg_022.txt     
  inflating: labels/Aimg_023.txt     
  inflating: labels/Aimg_024.txt     
  inflating: labels/Aimg_039.txt     
  inflating: labels/Aimg_041.txt     
  inflating: labels/Aimg_042.txt     
  inflating: labels/Aimg_043.txt     
  inflating: labels/Aimg_046.tx

In [9]:
%cd ../MidasYoloPlanercnn
!ls
!pwd

/content/MidasYoloPlanercnn
cfg					 models		     train.py	 wiki
data					 README.md	     transforms
dataloader				 testing_modules.py  utils
EVA5_Assignment15_MidasYolo_Train.ipynb  test.py	     weights
/content/MidasYoloPlanercnn


In [10]:
# Copy dataset
# Delete existing images and labels, and copy new ones from own gdrive folder
!rm data/customdata/images/*
!rm data/customdata/labels/*
!rm data/customdata/midas/*
# !rm data/customdata/planes/*

!mv /content/rawdata/images/* data/customdata/images/
!mv /content/rawdata/labels/* data/customdata/labels/
!mv /content/rawdata/depth/* data/customdata/midas/
# !mv /content/rawdata/planes/* data/customdata/planes/


In [11]:
# Copy custom config files
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.names data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.data data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_train.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_test.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/train.shapes data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/test.shapes data/customdata/


In [12]:
# These files have a problem, label coordinate > 1. Manually fix or else delete label:

!rm data/customdata/labels/8c7a3576.txt
!rm data/customdata/labels/8c7b2016.txt
!rm data/customdata/labels/8c7d69b6.txt

In [13]:
# Imports
import torch
import torch.nn as nn
from torchvision.transforms import Compose
from torchsummary import summary

import cv2
import numpy as np
import math
import glob
import os
import sys
import re



In [15]:
!ls

cfg					 models		     train.py	 wiki
data					 README.md	     transforms
dataloader				 testing_modules.py  utils
EVA5_Assignment15_MidasYolo_Train.ipynb  test.py	     weights


### **Model Summary**

In [17]:
from models.midasyolo3 import MidasYoloNet
IS_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if IS_CUDA else "cpu")

model = MidasYoloNet(yolo_cfg='cfg/yolov3-custom.cfg', device='cpu').to(device)


Loading weights:  None


Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth



Adding to routes from convolutional: 4
Adding to routes from convolutional: 16
Adding to routes from convolutional: 28
Model Summary: 60 layers, 1.61678e+07 parameters, 1.61678e+07 gradients


/content/MidasYoloPlanercnn/models/planercnn_net.py:458: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


In [18]:
print(model)

MidasYoloNet(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [21]:
model.info(True)

layer                                               name  gradient   parameters                shape         mu      sigma
    0 pretrained.layer1.0.weight                              True         9408        [64, 3, 7, 7]   -0.00056     0.0738
    1 pretrained.layer1.1.weight                              True           64                 [64]      0.988      0.592
    2 pretrained.layer1.1.bias                                True           64                 [64]       2.17       2.36
    3 pretrained.layer1.4.0.conv1.weight                      True        16384      [256, 64, 1, 1]   -0.00193     0.0265
    4 pretrained.layer1.4.0.bn1.weight                        True          256                [256]      0.747      0.434
    5 pretrained.layer1.4.0.bn1.bias                          True          256                [256]      0.137        0.8
    6 pretrained.layer1.4.0.conv2.weight                      True        18432       [256, 8, 3, 3]   0.000369     0.0222
    7 pretrained

In [ ]:
# from torchsummary import summary
# def show_model_summary(model, input_size):
#     result = summary(model, input_size=input_size)
#     print(result)

# show_model_summary(model.to('cuda'), (3, 416, 416))    

### **Training Yolo, with Encoder frozen**

Midas encoder and decoder are both frozen

#### **Training with initial weights**
Image size = 64, batch size = 16, epochs=100  
Best mAP Value: 0.155

In [20]:

!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 100 --load-init --img-size 64 --nosave



Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=100, evolve=False, freeze_encoder=False, img_size=[64], initWeights=True, init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=True, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 07:27:03.417470: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Downloading: "https://download.pytorch.org/models/ig

In [22]:
# Died at 50, but don't have resume. So running for 50 more from saved best weights
!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --weights weights/best.pt --img-size 64 --nosave


Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=50, evolve=False, freeze_encoder=False, img_size=[64], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 08:50:40.289226: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolution

In [24]:
! cp weights/best.pt /content/gdrive/'My Drive'/eva5/ass13/custom/weights/best15_a.pt
! cp weights/last.pt /content/gdrive/'My Drive'/eva5/ass13/custom/weights/last15_a.pt

#### **Training with prev weights - Image size 128**
Image size=128, batch size=16, epochs=100 (this run was interrupted)
Best mAP: 0.363

In [26]:
!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 100 --weights weights/best.pt --img-size 128 --nosave



Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=100, evolve=False, freeze_encoder=False, img_size=[128], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 10:53:28.969748: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convoluti

####**Trainig with Image size 256**
epochs=30
Best mAP: 0.498

In [27]:
!python train.py --data data/customdata/custom.data --batch 8 --cache --cfg cfg/yolov3-custom.cfg --epochs 30 --weights weights/best.pt --img-size 256 --nosave


Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=30, evolve=False, freeze_encoder=False, img_size=[256], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 12:14:12.196652: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolution

### Training Yolo, With Encoder Unfrozen
This is probably not very accurate, ignore


In [33]:
# Unfreeze encoder...
!python train.py --data data/customdata/custom.data --batch 4 --cache --cfg cfg/yolov3-custom.cfg --epochs 10 --weights weights/best-256.pt --img-size 512 --nosave --run-branches yolo,encoder


Namespace(accumulate=4, adam=False, batch_size=4, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=10, evolve=False, freeze_encoder=False, img_size=[512], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best-256.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 14:38:45.366069: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolu

#### Repeating above test with lesser image size

In [ ]:
!python train.py --data data/customdata/custom.data --batch 4 --cache --cfg cfg/yolov3-custom.cfg --epochs 10 --weights weights/best15_a.pt --img-size 64 --nosave --run-branches yolo,encoder


Namespace(accumulate=4, adam=False, batch_size=4, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=10, evolve=False, freeze_encoder=False, img_size=[64], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, run_branches='yolo,encoder', single_cls=False, weights='weights/best15_a.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 22:43:00.490190: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
A

### **Only Midas Encoder and Decoder**
Running without yolo layers, epoch=50